<a href="https://colab.research.google.com/github/zmkarakas/data_science_project_pyspark-plotly/blob/main/data_science_project_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
#to view this project, get the kaggle.json file from Kaggle and upload it to colaboratory, then run all the cells.

!pip install  --upgrade plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d arjunprasadsarkhel/2021-olympics-in-tokyo
!unzip 2021-olympics-in-tokyo.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.8/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.8/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open 2021-olympics-in-tokyo.zip, 2021-olympics-in-tokyo.zip.zip or 2021-olympics-in-tokyo.zip.ZIP.


In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 50.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=539399bb122773806473937df54057d771b67c17c868b23e020fe1526ea447b7
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [ ]:
import pandas as pd

def excel_to_csv(name):
  pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep = ',', index = False)

excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')


In [ ]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entriesGender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [ ]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [ ]:
medals.sort_values('Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
92,86,Syrian Arab Republic,0,0,1,1,77
81,77,Turkmenistan,0,1,0,1,77
79,77,North Macedonia,0,1,0,1,77
78,77,Lithuania,0,1,0,1,77
77,77,Saudi Arabia,0,1,0,1,77


In [ ]:
import plotly.express as px
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'], color_discrete_sequence =['Gold', 'Silver', 'Brown'], title='Number of Medals Per Country')
fig.show()

In [ ]:
coaches.head(5)

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [ ]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [ ]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as Count from coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(7)

,NOC,Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5


In [ ]:
fig = px.bar(pd_coaches_query, x= 'NOC', y= 'Count', color='Count', title='Number of Coaches from each country')
fig.show()

In [ ]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show(20)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [ ]:
spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').show(20)

+-----------+-------------------+-----+
|        NOC|         Discipline|Count|
+-----------+-------------------+-----+
|Afghanistan|          Athletics|    2|
|Afghanistan|           Shooting|    1|
|Afghanistan|           Swimming|    1|
|Afghanistan|          Taekwondo|    1|
|    Albania|Artistic Gymnastics|    1|
|    Albania|          Athletics|    1|
|    Albania|               Judo|    1|
|    Albania|           Shooting|    1|
|    Albania|           Swimming|    2|
|    Albania|      Weightlifting|    2|
|    Algeria|          Athletics|    8|
|    Algeria|             Boxing|    8|
|    Algeria|       Canoe Sprint|    1|
|    Algeria|       Cycling Road|    2|
|    Algeria|            Fencing|    3|
|    Algeria|               Judo|    2|
|    Algeria|  Marathon Swimming|    1|
|    Algeria|             Rowing|    2|
|    Algeria|            Sailing|    2|
|    Algeria|           Shooting|    1|
+-----------+-------------------+-----+
only showing top 20 rows



In [ ]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline,
             count(*) as Count from athletes group by NOC, Discipline
             order by NOC, Discipline''').toPandas()

In [ ]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline')
fig.show()

In [ ]:
athletes['Discipline'].value_counts().head(10)

Athletics       2068
Swimming         743
Football         567
Rowing           496
Hockey           406
Judo             373
Handball         343
Shooting         342
Sailing          336
Rugby Sevens     283
Name: Discipline, dtype: int64

In [ ]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming', 'Football'])]
pd_athletes_query.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [ ]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes  from each country per discipline')
fig.show()

In [ ]:
entriesGender.head(5)

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [ ]:
len(entriesGender)

46

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

specs = [[{'type':'domain'}, {'type':'domain'}]] * 23
fig = make_subplots(rows=23, cols=2, 
                    subplot_titles=entriesGender['Discipline'], specs=specs)
sub_figs= []
for index, row in entriesGender.iterrows():
  sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'],
                                                      row['Male']])
  sub_figs.append(sub_fig)

k = 0
for i in range(1,24):
  for j in range(1,3):
    fig.add_trace(sub_figs[k], i , j)
    k += 1

fig.update_layout(showlegend=False, height=9000, width=600, title_text="Distribution of gender amongst discipline")
fig.update_traces(textposition='inside', textinfo='label+percent',hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()